# Dependences

In [307]:
include("../../src/struct_data.jl")

fusion_agg (generic function with 1 method)

# Input Data

In [308]:
# X = Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
# Y = Float64.(readdlm("../../data/init/Sphere/10.0.xyz")[3:end,2:end]) |> cu

# Force Functions

## Cubic

In [309]:
@make_struct_func Cubic
dump(Cubic)

UnionAll
  var: TypeVar
    name: Symbol T
    lb: Union{}
    ub: Any
  body: Cubic{T} <: ForceType
    μ₁::T
    rₘᵢₙ::T
    rₘₐₓ::T


In [310]:
# Parameters
μ₁   = 1.0
rₘₐₓ = 2.0
rₘᵢₙ = 3.0
fₚ   = 0.1

0.1

In [311]:
Cubic(μ₁,rₘₐₓ,rₘᵢₙ)

Cubic{Float64}(1.0, 2.0, 3.0)

In [312]:
# Show function on specific r value
r = 1
force_func(Cubic(μ₁,rₘₐₓ,rₘᵢₙ),r)

4.0

In [313]:
dump(
        InteractionPar(
        Cubic(μ₁,rₘₐₓ,rₘᵢₙ),
        ContractilePar(fₚ)
    )
)

InteractionPar
  Force: Cubic{Float64}
    μ₁: Float64 1.0
    rₘᵢₙ: Float64 2.0
    rₘₐₓ: Float64 3.0
  Contractile: ContractilePar
    fₚ: Float64 0.1


## LennardJones

In [314]:
@make_struct_func LennardJones
dump(LennardJones)

UnionAll
  var: TypeVar
    name: Symbol T
    lb: Union{}
    ub: Any
  body: LennardJones{T} <: ForceType
    μ₁::T
    rₘᵢₙ::T
    rₘₐₓ::T


In [315]:
# Parameters
μ₁   = 1.0
rₘₐₓ = 2.0
rₘᵢₙ = 3.0

3.0

In [316]:
LennardJones(μ₁,rₘₐₓ,rₘᵢₙ)

LennardJones{Float64}(1.0, 2.0, 3.0)

In [317]:
# Show function on specific r value
r = 1.9
force_func(LennardJones(μ₁,rₘₐₓ,rₘᵢₙ),r)

1.9609746568876094

In [318]:
dump(
    InteractionPar(
        LennardJones(μ₁,rₘₐₓ,rₘᵢₙ),
        ContractilePar(fₚ)
    )
)

InteractionPar
  Force: LennardJones{Float64}
    μ₁: Float64 1.0
    rₘᵢₙ: Float64 2.0
    rₘₐₓ: Float64 3.0
  Contractile: ContractilePar
    fₚ: Float64 0.1


# Model Parameter

In [319]:
Model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 10000.0,
        dt    = 0.5,
        nₖₙₙ  = 50,
        nₛₐᵥₑ = 100
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input = "../../data/init/Sphere"
    )
)
dump(Model)

ModelSet
  Time: TimeModel
    tₛᵢₘ: Float64 10000.0
    dt: Float64 0.5
    nₖₙₙ: Int64 50
    nₛₐᵥₑ: Int64 100
  Input: InputModel
    outer_ratio: Float64 0.8
    path_input: String "../../data/init/Sphere"


# Aggregate

## Aggregate

In [320]:
Agg = Aggregate(
    "A",
    InteractionPar(
        Cubic(1.0,2.0,3.0), 
        ContractilePar(0.01)
    ),
    X,Model
)
show_aggregates(Agg)

------------------ Agg Properties -------------------
Name    = A
Radius  = 15.270000457763672
InteractionPar
  Force: Cubic{Float64}
    μ₁: Float64 1.0
    rₘᵢₙ: Float64 2.0
    rₘₐₓ: Float64 3.0
  Contractile: ContractilePar
    fₚ: Float64 0.01
--------------------- Position ----------------------


2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -1.5  -4.62  -13.88
  0.5  -4.62  -13.88
  2.5  -4.62  -13.88
 -4.5  -2.89  -13.88
 -2.5  -2.89  -13.88
 -0.5  -2.89  -13.88
  1.5  -2.89  -13.88
  3.5  -2.89  -13.88
 -5.5  -1.15  -13.88
 -3.5  -1.15  -13.88
 -1.5  -1.15  -13.88
  0.5  -1.15  -13.88
  2.5  -1.15  -13.88
  ⋮           
 -1.5   1.15   13.88
  0.5   1.15   13.88
  2.5   1.15   13.88
  4.5   1.15   13.88
 -4.5   2.89   13.88
 -2.5   2.89   13.88
 -0.5   2.89   13.88
  1.5   2.89   13.88
  3.5   2.89   13.88
 -1.5   4.62   13.88
  0.5   4.62   13.88
  2.5   4.62   13.88

--------------------- Outline -----------------------


1×2504 adjoint(::Vector{Int64}) with eltype Int64:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

Outer/Total = 0.46325878594249204


## Simple Aggregate

In [321]:
@time AllAggs = AllAggregates(
    [Aggregate("A", InteractionPar(Cubic(1.0,2.0,3.0), ContractilePar(0.01)), X, Model)]
    ,
    [
        ["A",[100 ,0   ,0  ]],
        ["A",[0   ,100 ,0  ]]
    ]
)
show_aggregates(AllAggs)

  0.391120 seconds (663.60 k allocations: 38.462 MiB, 97.74% compilation time)
---------------- Aggs Type List ---------------------


1×4 Matrix{Any}:
 1  …  InteractionPar(Cubic{Float64}(1.0, 2.0, 3.0), ContractilePar(0.01))

------------------- Aggs Index ----------------------
Index of List of Aggregates


1×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

Index of Number of Aggregates


...

--------------------- Position ----------------------


5008×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  98.5   -4.62  -13.88
 100.5   -4.62  -13.88
 102.5   -4.62  -13.88
  95.5   -2.89  -13.88
  97.5   -2.89  -13.88
  99.5   -2.89  -13.88
 101.5   -2.89  -13.88
 103.5   -2.89  -13.88
  94.5   -1.15  -13.88
  96.5   -1.15  -13.88
  98.5   -1.15  -13.88
 100.5   -1.15  -13.88
 102.5   -1.15  -13.88
   ⋮            
  -1.5  101.15   13.88
   0.5  101.15   13.88
   2.5  101.15   13.88
   4.5  101.15   13.88
  -4.5  102.89   13.88
  -2.5  102.89   13.88
  -0.5  102.89   13.88
   1.5  102.89   13.88
   3.5  102.89   13.88
  -1.5  104.62   13.88
   0.5  104.62   13.88
   2.5  104.62   13.88

--------------------- Outline -----------------------


1×5008 adjoint(::Vector{Int64}) with eltype Int64:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

Outer/Total = 0.46325878594249204


## Complex Aggregate

In [322]:
@time AllAggs = AllAggregates(
    [Aggregate("A", InteractionPar(Cubic(1.0,2.0,3.0), ContractilePar(0.01)), X, Model),
     Aggregate("B", InteractionPar(Cubic(2.0,2.0,3.0), ContractilePar(0.02)), Y, Model),
     Aggregate("C", InteractionPar(Cubic(3.0,2.0,3.0), ContractilePar(0.02)), X, Model),
     Aggregate("D", InteractionPar(Cubic(4.0,2.0,3.0), ContractilePar(0.04)), Y, Model)
    ],
    [
        ["A",[100 ,0   ,0  ]],
        ["C",[0   ,100 ,0  ]],
        ["D",[0   ,0   ,100]],
        ["B",[0   ,100 ,0  ]]
    ]
)
show_aggregates(AllAggs)

  0.013034 seconds (123.39 k allocations: 8.295 MiB)
---------------- Aggs Type List ---------------------


4×4 Matrix{Any}:
 1  …  InteractionPar(Cubic{Float64}(1.0, 2.0, 3.0), ContractilePar(0.01))
 2     InteractionPar(Cubic{Float64}(2.0, 2.0, 3.0), ContractilePar(0.02))
 3     InteractionPar(Cubic{Float64}(3.0, 2.0, 3.0), ContractilePar(0.02))
 4     InteractionPar(Cubic{Float64}(4.0, 2.0, 3.0), ContractilePar(0.04))

------------------- Aggs Index ----------------------
Index of List of Aggregates


1×6500 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  2  2  2  2  2  2  2  2  2  2  2  2

Index of Number of Aggregates


...

--------------------- Position ----------------------


6500×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  98.5   -4.62  -13.88
 100.5   -4.62  -13.88
 102.5   -4.62  -13.88
  95.5   -2.89  -13.88
  97.5   -2.89  -13.88
  99.5   -2.89  -13.88
 101.5   -2.89  -13.88
 103.5   -2.89  -13.88
  94.5   -1.15  -13.88
  96.5   -1.15  -13.88
  98.5   -1.15  -13.88
 100.5   -1.15  -13.88
 102.5   -1.15  -13.88
   ⋮            
   3.5   97.69    8.98
  -3.5   99.42    8.98
  -1.5   99.42    8.98
   0.5   99.42    8.98
   2.5   99.42    8.98
  -2.5  101.15    8.98
  -0.5  101.15    8.98
   1.5  101.15    8.98
   3.5  101.15    8.98
  -1.5  102.89    8.98
   0.5  102.89    8.98
   2.5  102.89    8.98

--------------------- Outline -----------------------


1×6500 adjoint(::Vector{Int64}) with eltype Int64:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

Outer/Total = 0.4523076923076923


## Fusion Aggs

### First

In [323]:
AGG = Aggregate(
    "HEK", 
    InteractionPar(
        Cubic(1.0,2.0,3.0), 
        ContractilePar(0.01)
    ),
    X, Model
)
println("===================================== One Aggregate =====================================")
show_aggregates(AGG)

===================================== One Aggregate =====================================
------------------ Agg Properties -------------------
Name    = HEK
Radius  = 15.270000457763672
InteractionPar
  Force: Cubic{Float64}
    μ₁: Float64 1.0
    rₘᵢₙ: Float64 2.0
    rₘₐₓ: Float64 3.0
  Contractile: ContractilePar
    fₚ: Float64 0.01
--------------------- Position ----------------------


2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -1.5  -4.62  -13.88
  0.5  -4.62  -13.88
  2.5  -4.62  -13.88
 -4.5  -2.89  -13.88
 -2.5  -2.89  -13.88
 -0.5  -2.89  -13.88
  1.5  -2.89  -13.88
  3.5  -2.89  -13.88
 -5.5  -1.15  -13.88
 -3.5  -1.15  -13.88
 -1.5  -1.15  -13.88
  0.5  -1.15  -13.88
  2.5  -1.15  -13.88
  ⋮           
 -1.5   1.15   13.88
  0.5   1.15   13.88
  2.5   1.15   13.88
  4.5   1.15   13.88
 -4.5   2.89   13.88
 -2.5   2.89   13.88
 -0.5   2.89   13.88
  1.5   2.89   13.88
  3.5   2.89   13.88
 -1.5   4.62   13.88
  0.5   4.62   13.88
  2.5   4.62   13.88

--------------------- Outline -----------------------


1×2504 adjoint(::Vector{Int64}) with eltype Int64:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

Outer/Total = 0.46325878594249204


In [324]:
println("=================================== Fusion Aggregate ====================================")
FusionAGG = fusion_agg(AGG)
show_aggregates(FusionAGG)

TEXT = hcat(FusionAGG.Outline,Matrix(FusionAGG.Position))
open("tetero.xyz", "w") do f
    write(f, "$(size(FusionAGG.Position, 1))\n")
    write(f, "t=$(0)\n")
    writedlm(f,TEXT, ' ')
end

=================================== Fusion Aggregate ====================================
---------------- Aggs Type List ---------------------


1×4 Matrix{Any}:
 1  …  InteractionPar(Cubic{Float64}(1.0, 2.0, 3.0), ContractilePar(0.01))

------------------- Aggs Index ----------------------
Index of List of Aggregates


1×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

Index of Number of Aggregates


...

--------------------- Position ----------------------


5008×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -16.77  -4.62  -13.88
 -14.77  -4.62  -13.88
 -12.77  -4.62  -13.88
 -19.77  -2.89  -13.88
 -17.77  -2.89  -13.88
 -15.77  -2.89  -13.88
 -13.77  -2.89  -13.88
 -11.77  -2.89  -13.88
 -20.77  -1.15  -13.88
 -18.77  -1.15  -13.88
 -16.77  -1.15  -13.88
 -14.77  -1.15  -13.88
 -12.77  -1.15  -13.88
   ⋮            
  13.77   1.15   13.88
  15.77   1.15   13.88
  17.77   1.15   13.88
  19.77   1.15   13.88
  10.77   2.89   13.88
  12.77   2.89   13.88
  14.77   2.89   13.88
  16.77   2.89   13.88
  18.77   2.89   13.88
  13.77   4.62   13.88
  15.77   4.62   13.88
  17.77   4.62   13.88

--------------------- Outline -----------------------


1×5008 adjoint(::Vector{Int64}) with eltype Int64:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

Outer/Total = 0.46325878594249204


### Second

In [325]:
AGG_1 = Aggregate(
    "HEK_1", 
    InteractionPar(
        Cubic(1.0,2.0,3.0), 
        ContractilePar(0.01)
    ),
    X, Model
)
show_aggregates(AGG_1)

------------------ Agg Properties -------------------
Name    = HEK_1
Radius  = 15.270000457763672
InteractionPar
  Force: Cubic{Float64}
    μ₁: Float64 1.0
    rₘᵢₙ: Float64 2.0
    rₘₐₓ: Float64 3.0
  Contractile: ContractilePar
    fₚ: Float64 0.01
--------------------- Position ----------------------


2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -1.5  -4.62  -13.88
  0.5  -4.62  -13.88
  2.5  -4.62  -13.88
 -4.5  -2.89  -13.88
 -2.5  -2.89  -13.88
 -0.5  -2.89  -13.88
  1.5  -2.89  -13.88
  3.5  -2.89  -13.88
 -5.5  -1.15  -13.88
 -3.5  -1.15  -13.88
 -1.5  -1.15  -13.88
  0.5  -1.15  -13.88
  2.5  -1.15  -13.88
  ⋮           
 -1.5   1.15   13.88
  0.5   1.15   13.88
  2.5   1.15   13.88
  4.5   1.15   13.88
 -4.5   2.89   13.88
 -2.5   2.89   13.88
 -0.5   2.89   13.88
  1.5   2.89   13.88
  3.5   2.89   13.88
 -1.5   4.62   13.88
  0.5   4.62   13.88
  2.5   4.62   13.88

--------------------- Outline -----------------------


1×2504 adjoint(::Vector{Int64}) with eltype Int64:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

Outer/Total = 0.46325878594249204


In [326]:
AGG_2 = Aggregate(
    "HEK_2", 
    InteractionPar(
        Cubic(1.0,2.0,3.0), 
        ContractilePar(0.01)
    ),
    Y, Model
)
show_aggregates(AGG_2)

------------------ Agg Properties -------------------
Name    = HEK_2
Radius  = 10.430000305175781
InteractionPar
  Force: Cubic{Float64}
    μ₁: Float64 1.0
    rₘᵢₙ: Float64 2.0
    rₘₐₓ: Float64 3.0
  Contractile: ContractilePar
    fₚ: Float64 0.01
--------------------- Position ----------------------


746×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -1.5  -2.89  -8.98
  0.5  -2.89  -8.98
  2.5  -2.89  -8.98
 -2.5  -1.15  -8.98
 -0.5  -1.15  -8.98
  1.5  -1.15  -8.98
  3.5  -1.15  -8.98
 -3.5   0.58  -8.98
 -1.5   0.58  -8.98
  0.5   0.58  -8.98
  2.5   0.58  -8.98
 -2.5   2.31  -8.98
 -0.5   2.31  -8.98
  ⋮           
  3.5  -2.31   8.98
 -3.5  -0.58   8.98
 -1.5  -0.58   8.98
  0.5  -0.58   8.98
  2.5  -0.58   8.98
 -2.5   1.15   8.98
 -0.5   1.15   8.98
  1.5   1.15   8.98
  3.5   1.15   8.98
 -1.5   2.89   8.98
  0.5   2.89   8.98
  2.5   2.89   8.98

--------------------- Outline -----------------------


1×746 adjoint(::Vector{Int64}) with eltype Int64:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

Outer/Total = 0.4155495978552279


In [327]:
Fusion_AGG = AllAggregates(
        [AGG_1, AGG_2],
        [
            [AGG_1.Name,[-AGG_1.Radius ,0 ,0]],
            [AGG_2.Name,[ AGG_2.Radius ,0 ,0]]
        ]
    )
    
TEXT = hcat(Fusion_AGG.Outline,Matrix(Fusion_AGG.Position))
open("tetero_2.xyz", "w") do f
    write(f, "$(size(Fusion_AGG.Position, 1))\n")
    write(f, "t=$(0)\n")
    writedlm(f,TEXT, ' ')
end

### Third

In [328]:
AGG_1 = Aggregate(
    "HEK_1", 
    InteractionPar(
        Cubic(1.0,2.0,3.0), 
        ContractilePar(0.01)
    ),
    X, Model
)
show_aggregates(AGG_1)

------------------ Agg Properties -------------------
Name    = HEK_1
Radius  = 15.270000457763672
InteractionPar
  Force: Cubic{Float64}
    μ₁: Float64 1.0
    rₘᵢₙ: Float64 2.0
    rₘₐₓ: Float64 3.0
  Contractile: ContractilePar
    fₚ: Float64 0.01
--------------------- Position ----------------------


2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -1.5  -4.62  -13.88
  0.5  -4.62  -13.88
  2.5  -4.62  -13.88
 -4.5  -2.89  -13.88
 -2.5  -2.89  -13.88
 -0.5  -2.89  -13.88
  1.5  -2.89  -13.88
  3.5  -2.89  -13.88
 -5.5  -1.15  -13.88
 -3.5  -1.15  -13.88
 -1.5  -1.15  -13.88
  0.5  -1.15  -13.88
  2.5  -1.15  -13.88
  ⋮           
 -1.5   1.15   13.88
  0.5   1.15   13.88
  2.5   1.15   13.88
  4.5   1.15   13.88
 -4.5   2.89   13.88
 -2.5   2.89   13.88
 -0.5   2.89   13.88
  1.5   2.89   13.88
  3.5   2.89   13.88
 -1.5   4.62   13.88
  0.5   4.62   13.88
  2.5   4.62   13.88

--------------------- Outline -----------------------


1×2504 adjoint(::Vector{Int64}) with eltype Int64:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

Outer/Total = 0.46325878594249204


In [329]:
AGG_2 = Aggregate(
    "HEK_2", 
    InteractionPar(
        Cubic(1.0,2.0,3.0), 
        ContractilePar(0.01)
    ),
    Y, Model
)
show_aggregates(AGG_2)

------------------ Agg Properties -------------------
Name    = HEK_2
Radius  = 10.430000305175781
InteractionPar
  Force: Cubic{Float64}
    μ₁: Float64 1.0
    rₘᵢₙ: Float64 2.0
    rₘₐₓ: Float64 3.0
  Contractile: ContractilePar
    fₚ: Float64 0.01
--------------------- Position ----------------------


746×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -1.5  -2.89  -8.98
  0.5  -2.89  -8.98
  2.5  -2.89  -8.98
 -2.5  -1.15  -8.98
 -0.5  -1.15  -8.98
  1.5  -1.15  -8.98
  3.5  -1.15  -8.98
 -3.5   0.58  -8.98
 -1.5   0.58  -8.98
  0.5   0.58  -8.98
  2.5   0.58  -8.98
 -2.5   2.31  -8.98
 -0.5   2.31  -8.98
  ⋮           
  3.5  -2.31   8.98
 -3.5  -0.58   8.98
 -1.5  -0.58   8.98
  0.5  -0.58   8.98
  2.5  -0.58   8.98
 -2.5   1.15   8.98
 -0.5   1.15   8.98
  1.5   1.15   8.98
  3.5   1.15   8.98
 -1.5   2.89   8.98
  0.5   2.89   8.98
  2.5   2.89   8.98

--------------------- Outline -----------------------


1×746 adjoint(::Vector{Int64}) with eltype Int64:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

Outer/Total = 0.4155495978552279


In [330]:
Fusion_AGG_2 = AllAggregates(
        [AGG_1, AGG_2],
        [
            [AGG_1.Name,[0                           , 0                           ,0]],
            [AGG_2.Name,[AGG_2.Radius + AGG_1.Radius ,0                            ,0]],
            [AGG_2.Name,[0                           ,AGG_2.Radius + AGG_1.Radius  ,0]]
        ]
    )

TEXT = hcat(Fusion_AGG_2.Outline,Matrix(Fusion_AGG_2.Position))
open("tetero_3.xyz", "w") do f
    write(f, "$(size(Fusion_AGG_2.Position, 1))\n")
    write(f, "t=$(0)\n")
    writedlm(f,TEXT, ' ')
end

# Simulation Struct

In [331]:
SSS = SimulationSet(Fusion_AGG_2, Model)
show_simulation_set(SSS)

============== Type of Matrix =============
CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}
========== Neighbors Matrix Size ==========
idx      = (3996, 3996)
idx_red  = (21, 3996)
idx_sum  = (1, 3996)
idx_cont = (50, 3996)
============ Forces Matrix Size ===========
dX       = (3996, 3)
F       = (3996, 3)
